# Carga de las librerías necesarias
Se importan las librerías necesarias para la ejecución del algoritmo.

Las librerías deben estar previamente instaladas en Python.

In [7]:
%reset -f

import numpy as np
import pandas as pd
import dateutil

# Lectura de datos
Función para leer los archivos de datos necesarios para ejecutar el algoritmo y preparar los datos para su ejecución, es decir, construir los campos adicionales requeridos.

El archivo debe contener los siguientes campos en ese orden:
* ID_Transformador
* Fecha
* Voltaje_F1_V
* Voltaje_F2_V
* Voltaje_F3_V
* Corriente_F1_A
* Corriente_F2_A
* Corriente_F3_A
* Factor_Potencia
* Energia_Activa_kWh	
* Energia_Reactiva_kVAR
* Potencia_Activa_kW
* Potencia_Reactiva_kVAR
* Potencia_Aparente_kVA

El separador de decimales debe ser punto.

In [8]:
def load_data():

    global data
    
    data = pd.read_csv("datos.csv", sep=',', decimal='.')
    
    # eliminar las comas
    data.columns = [col.replace(',','') for col in data.columns]
    
    # construir columnas necesarias para el análisis
    data['Fecha'] = data['Fecha'].apply(dateutil.parser.parse, dayfirst=True)
    data['ano'] = pd.DatetimeIndex(data['Fecha']).year
    data['mes'] = pd.DatetimeIndex(data['Fecha']).month
    data['dia'] = pd.DatetimeIndex(data['Fecha']).day
    data['hora'] = pd.DatetimeIndex(data['Fecha']).hour
    data['minuto'] = pd.DatetimeIndex(data['Fecha']).minute
    data = data.sort_values(['ID_Transformador', 'Fecha'], ascending=[True, True])
    data['carga'] = data['Energia_Activa_kWh'] - data['Energia_Activa_kWh'].shift(1) 
    data.loc[(data.carga < 0), 'carga'] = np.nan
    data.loc[(data.hora != data.hora.shift(1)) & (data.hora - 1 != data.hora.shift(1)), 'carga'] = np.nan
    data.loc[(data.dia != data.dia.shift(1)) & (data.dia - 1 != data.dia.shift(1)), 'carga'] = np.nan
    
    # remover outliers
    # Los outliers se presentan por inconsistencias en la informacion.
    # Outlier: dato que esta a mas de 2.5 desviaciones estandar de la media.
    grupos = data.groupby(['ID_Transformador'])
    prov = pd.DataFrame(grupos['carga'].mean())
    prov['LS'] = grupos['carga'].mean() + grupos['carga'].std() * 2.5
    prov['LI'] = grupos['carga'].mean() - grupos['carga'].std() * 2.5
    prov['id_t'] = prov.index
    data = data.rename(columns={'ID_Transformador': 'id_t'})
    data = data.merge(prov[['LS','LI','id_t']], on = 'id_t',how='right')
    data.loc[(data.carga >= data.LS) | (data.carga <= data.LI), 'carga'] = np.nan    

# Calcular carga máxima y carga promedio por nodo
Función para calcular la carga maxima y la carga promedio de cada nodo por dia
* carga máxima por día: maxima de las cargas consolidadas por hora.
* carga promedio por día: promedio de las cargas consolidadas por hora.
* carga máxima por modo: promedio de las cargas maximas por dia.
* carga promedio por nodo: promedio de las cargas promedio por dia.

In [9]:
def cmaxpromnodo():

    global c_max_prom_nodo
    
    grupos1 = data.groupby(['id_t','ano','mes','dia','hora'])
    prov1 = pd.DataFrame(grupos1['carga'].sum())

    # cálculos por nodo, por día
    grupos2 = prov1.groupby(['id_t','ano','mes','dia'])
    c_max_prom_dia = pd.DataFrame(grupos2['carga'].max())
    c_max_prom_dia['cmax'] = grupos2['carga'].max()
    c_max_prom_dia['cpro'] = grupos2['carga'].mean()
    
    # consolidados por nodo
    grupos = c_max_prom_dia.groupby(['id_t','ano'])
    c_max_prom_nodo = pd.DataFrame(grupos['cmax'].max())
    c_max_prom_nodo['cmax'] = grupos['cmax'].mean()
    c_max_prom_nodo['prom'] = grupos['cpro'].mean()

# Calcular demanda promedio por nodo
Función para calcular la demanda promedio de cada nodo, como el promedio de las demandas por día

In [10]:
def ddapromnodo():

    global d_prom_nodo
    
    grupos = data.groupby(['id_t','ano','mes','dia'])
    prov = pd.DataFrame(grupos['carga'].sum())
    prov['horas_med'] = ((grupos['Fecha'].max() - grupos['Fecha'].min()).dt.total_seconds())/(60*60)
    prov['dda_med'] = grupos['Energia_Activa_kWh'].max() - grupos['Energia_Activa_kWh'].min()
    prov['dda_dia'] = prov['dda_med'] / prov['horas_med'] * 24
    
    grupos1 = prov.groupby(['id_t','ano'])
    d_prom_nodo = pd.DataFrame(grupos1['dda_dia'].mean())
    d_prom_nodo.index.names = ['id_tf','ano']
    d_prom_nodo['id_t'] = d_prom_nodo.index.get_level_values('id_tf')

# Run
Función para consolidar la informacion de carga promedio, carga maxima y demanda por nodo.

In [13]:
def run():
    cmaxpromnodo()
    ddapromnodo()
    cyd_pornodo = c_max_prom_nodo.merge(d_prom_nodo[['dda_dia','id_t']], on = 'id_t',how='right')
    
    # reporte en csv
    cyd_pornodo.to_csv('ResData/cyd_pornodo.csv')

# Ejecutar
Celda desde donde se ejecutan las dos rutinas esenciales:

* load_data(): carga de información
* run (): ejecución del algoritmo

En esta celda se especifican también los directorios de trabajo y la demanda con la cual se desea ejecutar la simulación.

In [14]:
load_data()
run()